##### Copyright 2018 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Spatial Attribution -- Building Blocks of Interpretability

This colab notebook is part of our **Building Blocks of Intepretability** series exploring how intepretability techniques combine together to explain neural networks. If you haven't already, make sure to look at the [**corresponding paper**](https://distill.pub/2018/building-blocks) as well!

This notebook demonstrates **Spatial Attribution**, a technique for exploring how detectors a different spatial positions in the network effected its output.

<br>

<img src="https://storage.googleapis.com/lucid-static/building-blocks/notebook_heroes/spatial-attribution.jpeg" width="648"></img>

<br>

This tutorial is based on [**Lucid**](https://github.com/tensorflow/lucid), a network for visualizing neural networks. Lucid is a kind of spiritual successor to DeepDream, but provides flexible abstractions so that it can be used for a wide range of interpretability research.

**Note**: The easiest way to use this tutorial is [as a colab notebook](), which allows you to dive in with no setup. We recommend you enable a free GPU by going:

> **Runtime**   →   **Change runtime type**   →   **Hardware Accelerator: GPU**

Thanks for trying Lucid!

#### **This notebook is a Jupyter version of the original Google Colab Notebook. This version adds widgets to facilitate the use of Lucid on your own images.**

# Install / Import / Load

This code depends on [Lucid](https://github.com/tensorflow/lucid) (our visualization library), and [svelte](https://svelte.technology/) (a web framework). The following cell will install both of them, and dependancies such as TensorFlow. And then import them as appropriate.

In [2]:
# !npm install -g svelte-cli@2.2.0
import os

import numpy as np
from ipyfilechooser import FileChooser
import ipywidgets as widgets
from IPython.display import display
from IPython.core.display import display, HTML

import tensorflow as tf
# uncomment to avoid deprecation warnings :
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
tf.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import lucid.modelzoo.vision_models as models
from lucid.misc.io import show
import lucid.optvis.render as render
from lucid.misc.io import show, load
from lucid.misc.io.reading import read
from lucid.misc.io.showing import _image_url
from lucid.misc.gradient_override import gradient_override_map
import lucid.scratch.web.svelte as lucid_svelte

# Attribution Code

In [3]:
model = models.InceptionV1()

In [4]:
labels_str = read("https://gist.githubusercontent.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57/raw/aa66dd9dbf6b56649fa3fab83659b2acbf3cbfd1/map_clsloc.txt",mode='r')
labels = [line[line.find(" "):].strip() for line in labels_str.split("\n")]
labels = [label[label.find(" "):].strip().replace("_", " ") for label in labels]
labels = sorted(["dummy"] + labels)

In [5]:
def raw_class_spatial_attr(model, img, layer, label, override=None):
    """
    How much did spatial positions at a given layer effect a output class?
    Returns:
      array containing attributions of layer 1 on layer2 where array[i,j] is
      the influence of layer1 on spatial posittion (i,j) of layer 2.
    """

    # Set up a graph for doing attribution...
    with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
        t_input = tf.placeholder_with_default(img, [None, None, 3])
        T = render.import_model(model, t_input, t_input)

        # Compute activations
        acts = T(layer).eval()

        if label is None: return np.zeros(acts.shape[1:-1])

        # Compute gradient between current layer and output score
        score = T("softmax2_pre_activation")[0, labels.index(label)]

        t_grad = tf.gradients([score], [T(layer)])[0] 
        grad = t_grad.eval({T(layer) : acts})

        # Linear approximation of effect of spatial position
        return np.sum(acts * grad, -1)[0]

In [6]:
def raw_spatial_spatial_attr(model, img, layer1, layer2, override=None):
    """Attribution between spatial positions in two different layers.
    """
    # Set up a graph for doing attribution...
    with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
        t_input = tf.placeholder_with_default(img, [None, None, 3])
        T = render.import_model(model, t_input, t_input)

        # Compute activations
        acts1 = T(layer1).eval()
        acts2 = T(layer2).eval({T(layer1) : acts1})

        # Construct gradient tensor
        # Backprop from spatial position (n_x, n_y) in layer2 to layer1.
        n_x, n_y = tf.placeholder("int32", []), tf.placeholder("int32", [])
        # channelwise magnitude of layer2 activation for each spatial position:
        layer2_mags = tf.sqrt(tf.reduce_sum(T(layer2)**2, -1))[0]
        score = layer2_mags[n_x, n_y]
        t_grad = tf.gradients([score], [T(layer1)])[0]

        # Compute attribution backwards from each position in layer2
        attrs = [] #
        for i in range(acts2.shape[1]):
            attrs_ = []
            for j in range(acts2.shape[2]):
                grad = t_grad.eval({n_x : i, n_y : j, T(layer1) : acts1})
                # linear approximation of impact (summed on channel dimension)
                attr = np.sum(acts1 * grad, -1)[0]
                attrs_.append(attr)
            attrs.append(attrs_)
        return np.asarray(attrs)

In [7]:
def orange_blue(a,b,clip=False):
    """
    Args:
    a: spatial position - class gradients array of shape (y_dim, x_dim) of the current layer for class A
    b: spatial position - class gradients array of shape (y_dim, x_dim) of the current layer for class B
    clip: whether to clip negative gradients for a and b
    
    Returns:
      Heatmap of the image for both classes A and B.
          Red channel is for A, Green channel is the mean influence of both classes
          and blue channel is for B.
    """
    
    if clip: # keeping positive values only
        a,b = np.maximum(a,0), np.maximum(b,0)
    arr = np.stack([a, (a + b)/2., b], -1)
    arr /= 1e-2 + np.abs(arr).max()/1.5
    arr += 0.3 
    return arr

# Spatial Attribution Interface

In this section, we build the *interface* for interacting with the different kinds of spatial attribution data that we can compute using the above functions. Feel free to skip over this if you aren't interested in that part. The main reason we're including it is so that you can change the interface if you want to.


In [8]:
%%html_define_svelte SpatialWidget

<div class="figure" style="width: 500px; height: 250px; contain: strict;">
  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint1 : spritemap1[pos1[1]][pos1[0]]}}">

    <svg class="pointer_container" viewBox="0 0 {{size1}} {{size1}}">
      {{#each xs1 as x}}
      {{#each ys1 as y}}
        <rect x={{x}} y={{y}} width=1 height=1
          class={{(pos2 != undefined && x == pos2[0] && y == pos2[1])? "selected" : ""}}
          on:mouseover="set({pos2: [x,y], pos1: undefined})"></rect>
      {{/each}}
      {{/each}}
    </svg> 

    <div class="label">{{layer1}}</div>
  </div>

  <div class="outer" on:mouseleave="set({pos1: undefined})">
    <img class="img" src="{{img}}">
    <img class="attr" src="{{(pos2 == undefined)? hint2 : spritemap2[pos2[1]][pos2[0]]}}">

    <svg class="pointer_container" viewBox="0 0 {{size2}} {{size2}}">
      {{#each xs2 as x}}
      {{#each ys2 as y}}
        <rect x={{x}} y={{y}} width=1 height=1
          class={{(pos1 != undefined && x == pos1[0] && y == pos1[1])? "selected" : ""}}
          on:mouseover="set({pos1: [x,y], pos2: undefined})"></rect>
      {{/each}}
      {{/each}}
    </svg> 

    <div class="label">{{layer2}}</div>
  </div>
  
</div>


<style>

  .outer{
    width: 224px;
    height: 224px;
    display: inline-block;
    margin-right: 2px;
    position: relative;
  }
  .img, .pointer_container {
    position: absolute;
    left: 0px;
    top: 0px;
    width: 224px;
    height: 224px;
    image-rendering: pixelated; 
  }
  .img {
      z-index: -100;
  }
  .attr {
    opacity: 0.6;
    position: absolute;
    left: 0px;
    top: 0px; {{#replace with -14px for Jupyter Classic}}
    width: 224px;
  }
  .outer .pointer_container  {
    z-index: 100;
  }
  .pointer_container rect {
    opacity: 0;
  }
  .pointer_container .selected  {
    opacity: 1;
    fill: none;
    stroke: hsl(24, 100%, 50%);
    stroke-width: 0.1px;
  }
  .label{
    position: absolute;
    left: 0px;
    top: 226px;
    width: 224px;
    color: black;
  }
</style>

<script>
  function range(n){
    return Array(n).fill().map((_, i) => i);
  }
  
  export default {
    data () {
      return {
        img: "",
        hint1: "",
        hint2: "",
        spritemap1 : "",
        size1: 1,
        spritemap2 : "",
        size2: 1,
        pos1: undefined,
        pos2: undefined,
        layer1: "",
        layer2: ""
      };
    },
    computed: {
      xs1: (size1) => range(size1),
      ys1: (size1) => range(size1),
      xs2: (size2) => range(size2),
      ys2: (size2) => range(size2)
    },
    helpers: {range}
  };
</script>

Trying to build svelte component from html...
svelte compile --format iife /tmp/svelte_0f883yoj/SpatialWidget_a5fb8c7f_0cb5_45d2_912b_6f11532bd357.html > /tmp/svelte_0f883yoj/SpatialWidget_a5fb8c7f_0cb5_45d2_912b_6f11532bd357.js
Svelte build failed! Output:
svelte version 1.64.1
compiling ../../../../../../../tmp/svelte_0f883yoj/SpatialWidget_a5fb8c7f_0cb5_45d2_912b_6f11532bd357.html...
Identifier is expected
62:     position: absolute;
63:     left: 0px;
64:     top: 0px; {{#replace with -14px for Jupyter Classic}}
                  ^
65:     width: 224px;
66:   }



In [9]:
def image_url_grid(grid):
    return [[_image_url(img) for img in line] for line in grid ]


def spatial_spatial_attr(model, img, layer1, layer2, hint_label_1=None, hint_label_2=None, override=None):
    hint1 = orange_blue(
      raw_class_spatial_attr(model, img, layer1, hint_label_1, override=override),
      raw_class_spatial_attr(model, img, layer1, hint_label_2, override=override),
      clip=True
    )
    hint2 = orange_blue(
      raw_class_spatial_attr(model, img, layer2, hint_label_1, override=override),
      raw_class_spatial_attr(model, img, layer2, hint_label_2, override=override),
      clip=True
    )

    attrs = raw_spatial_spatial_attr(model, img, layer1, layer2, override=override)
    attrs = attrs / attrs.max()

    lucid_svelte.SpatialWidget({
        "spritemap1": image_url_grid(attrs),
        "spritemap2": image_url_grid(attrs.transpose(2,3,0,1)),
        "size1": attrs.shape[3],
        "layer1": layer1,
        "size2": attrs.shape[0],
        "layer2": layer2,
        "img" : _image_url(img),
        "hint1": _image_url(hint1),
        "hint2": _image_url(hint2)
    })

# Attribution With GradPool override hack

In [10]:
def blur(x, w1, w2):
    """Spatially blur a 4D tensor."""
    x_ = tf.pad(x, [(0,0), (1,1), (1,1), (0,0)], "CONSTANT")
    x_jitter_hv = (x_[:, 2:, 1:-1] + x_[:, :-2, 1:-1] + x_[:, 1:-1, 2:] + x_[:, 1:-1, :-2])/4.
    x_jitter_diag = (x_[:, 2:, 2:] + x_[:, 2:, :-2] + x_[:, :-2, 2:] + x_[:, :-2, :-2])/4.
    return (1-w1-w2)*x + w1*x_jitter_hv + w2*x_jitter_diag

def make_MaxSmoothPoolGrad(blur_hack=False):
    """Create a relaxed version of the MaxPool gradient.

    GoogLeNet's use of MaxPooling creates a lot of gradient artifacts. This
    function creates a fake gradient that gets rid of them, reducing distractions
    in our UI demos.

    Be very very careful about using this in real life. It hides model behavior
    from you. This can help you see other things more clearly, but in most cases
    you probably should do something else.

    We're actively researching what's going on here.

    Args:
    blur_hack: If True, use the second less principled trick of slightly
      blurring the gradient to get rid of checkerboard artifacts.

    Returns:
    Gradient function.

    """
    def MaxPoolGrad(op, grad):
        inp = op.inputs[0]

        # Hack 1 (moderately principled): use a relaxation of the MaxPool grad
        # ---------------------------------------------------------------------
        #
        # Construct a pooling function where, if we backprop through it,
        # gradients get allocated proportional to the input activation.
        # Then backpropr through that instead.
        #
        # In some ways, this is kind of spiritually similar to SmoothGrad
        # (Smilkov et al.). To see the connection, note that MaxPooling introduces
        # a pretty arbitrary discontinuity to your gradient; with the right
        # distribution of input noise to the MaxPool op, you'd probably smooth out
        # to this. It seems like this is one of the most natural ways to smooth.
        #
        # We'll probably talk about this and related things in future work.

        op_args = [op.get_attr("ksize"), op.get_attr("strides"), op.get_attr("padding")]
        smooth_out = tf.nn.avg_pool(inp**2, *op_args)/ (1e-2+tf.nn.avg_pool(tf.abs(inp), *op_args))
        inp_smooth_grad = tf.gradients(smooth_out, [inp], grad)[0]

        # Hack 2 (if argument is set; not very principled) 
        # -------------------------------------------------
        #
        # Slightly blur gradient to get rid of checkerboard artifacts.
        # Note, this really isn't principled. We're working around / hiding a bad
        # property of the model. It should really be fixed by better model design.
        #
        # We do this so that the artifacts don't distract from the UI demo, but we
        # don't endorse people doing it in real applications.

        if blur_hack:
            inp_smooth_grad = blur(inp_smooth_grad, 0.5, 0.25)

        return inp_smooth_grad
    return MaxPoolGrad

In [11]:
def compare_attrs(model, img, layer1, layer2, hint_label_1, hint_label_2):
    print("Normal gradient:\n")

    spatial_spatial_attr(model, img, layer1, layer2,
                       hint_label_1=hint_label_1, hint_label_2=hint_label_2)

    print("\nSmooth MaxPool Grad:")
    print("note the subtle checkerboard patterns)\n")

    spatial_spatial_attr(model, img, layer1, layer2,
                       hint_label_1=hint_label_1, hint_label_2=hint_label_2,
                       override={"MaxPool": make_MaxSmoothPoolGrad()})

    print("\nSmooth + Blur MaxPool Grad:\n")

    spatial_spatial_attr(model, img, layer1, layer2,
                       hint_label_1=hint_label_1, hint_label_2=hint_label_2,
                       override={"MaxPool": make_MaxSmoothPoolGrad(blur_hack=True)})

In [12]:
print(
    "Upload file from local machine and select uploading path (A) or just select one file (B):"
)
print("A1) Select a file to upload")
uploader = widgets.FileUpload(accept='', multiple=False)
display(uploader)

print("\nA2) Select destination for uploaded file")
print("B) Select file in this server")
notebooks_root_path = ""
fc = FileChooser(".",
                 use_dir_icons=True,
                 select_default=True)
display(fc)


layers_list = [layer.name for layer in model.layers[3:]]
print("\nSelect the shallowest layer whose influence is being studied: ")
layers_widget = widgets.Dropdown(
    options=layers_list,
    value=layers_list[0],
    description='Layers'
)
display(layers_widget)

print("\nSelect the deepest layer whose influence is being studied: ")
layers_widget_bis = widgets.Dropdown(
    options=layers_list,
    value=layers_list[1],
    description='Layers'
)
display(layers_widget_bis)

print("\nSelect the first class  whose influence is being studied:: ")
classes_widget = widgets.Dropdown(
    options=labels,
    value=labels[0],
    description='Classes of documents'
)
display(classes_widget)

print("\nSelect the second class  whose influence is being studied:: ")
classes_widget_bis = widgets.Dropdown(
    options=labels,
    value=labels[1],
    description='Classes of documents'
)
display(classes_widget_bis)

Upload file from local machine and select uploading path (A) or just select one file (B):
A1) Select a file to upload


FileUpload(value={}, description='Upload')


A2) Select destination for uploaded file
B) Select file in this server


FileChooser(path='.', filename='', show_hidden='False')


Select the shallowest layer whose influence is being studied: 


...

Dropdown(description='Layers', index=1, options=('mixed3a', 'mixed3b', 'mixed4a', 'mixed4b', 'mixed4c', 'mixed…


Select the first class  whose influence is being studied:: 


Dropdown(description='Classes of documents', options=('Afghan hound', 'African chameleon', 'African crocodile'…


Select the second class  whose influence is being studied:: 


Dropdown(description='Classes of documents', index=1, options=('Afghan hound', 'African chameleon', 'African c…

In [13]:
if uploader.value: # upload local file to server
    picture_name = uploader.value[0]
    content = uploader.value[picture_name]['content'] # memoryview of the file
    picture_path = os.path.join(fc.selected_path, picture_name)
    with open(picture_name, 'wb') as f:
        f.write(content)
else: # use files already on the server
    picture_path = fc.default_filename
        
layer_name_1 = layers_widget.value # layers to use semantic dictionnary on
layer_name_2 = layers_widget_bis.value # layers to use semantic dictionnary on

class_name_1 = classes_widget.value # layers to use semantic dictionnary on
class_name_2 = classes_widget_bis.value # layers to use semantic dictionnary on

In [14]:
legend = "<h4> Legend : </h4>"
legend += "<div style='color:orange;font-weight: bold;'>%s</div>" % class_name_1
legend += "<div style='color:#85c1e9;font-weight: bold;'>%s</div>" % class_name_2
display(HTML(legend))

img = load(picture_path)

spatial_spatial_attr(model, img, layer_name_1, layer_name_2, hint_label_1=class_name_1, hint_label_2=class_name_2)

RuntimeError: No extension in URL: 